# Workshop: Sprachassistenten für offene Daten erstellen
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/freinold/odd24-workshop/blob/master/workshop.ipynb)

In diesem Workshop wollen wir 
- einen offenen Datensatz der Landeshauptstadt München erfassen und
- für ein KI-Sprachmodell verfügbar machen,
- sodass dieses uns dann Fragen dazu beantworten kann. 

In [ ]:
# Zu Beginn müssen wir die benötigten Bibliotheken installieren 

! pip install datasets langchain langchain-openai langchain-community faiss-cpu gradio SpeechRecognition gTTS

## Daten erhalten

Wir nutzen den Datensatz über die von der LHM angebotenen Dienstleistungen, welcher auf Huggingface gehostet wird:

https://opendata.muenchen.de/dataset/question-answering-datensatz-basierend-auf-den-dokumenten-des-muenchner-dienstleistungsfinders

In [ ]:
# Dataset von Huggingface herunterladen
# Tipp: Beim Laden noch den Parameter sample_by="document" hinzufügen, um die Dokumente als ganzes zu erhalten

from datasets import load_dataset

dataset = 

Wir können uns erst einmal Informationen über den Datensatz anzeigen lassen:

In [ ]:
print("Informationen über das Dataset:\n", dataset)

Nun müssen wir noch sicher gehen, dass unser Datensatz keine Duplikate enthält, aber alle Dokumente vorhanden sind:

In [ ]:
# Elemente deduplizieren und in Liste überführen
docs = [elem["text"] for elem in dataset["train"]]
docs = sorted(set(docs))

n_docs = 
first_doc = 
last_doc =

# Informationen zum Dataset ausgeben
print("Anzahl Dokumente:", n_docs)
print("Erstes Dokument:", first_doc)
print("Letztes Dokument:", last_doc)

Zuletzt können wir uns noch beispielhaft ein gesamtes Dokument ausgeben: 

In [ ]:
print("Ein gesamtes Dokument:\n", )

## KI-Sprachmodell aufrufen

Als nächstes wollen wir ein KI-Sprachmodell, in unserem Fall GPT-3.5 über Azure, aufrufen.
Hierzu brauchen wir einen API-Key, den ihr im Workshop erhaltet: https://yopad.eu/p/odd24ki

In [ ]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_version="2024-02-01",
    azure_deployment="gpt-35-turbo",
    api_key="", # Hier API-Key einfügen
    azure_endpoint="" # Hier Azure-Endpoint einfügen
)

Jetzt können wir GPT-3.5 schon erste Fragen stellen, indem wir die Funktion `llm.invoke()` aufrufen:

In [ ]:
frage = "" # Eigene Frage einfügen
antwort = 
print("Frage:", frage)
print("Antwort:", antwort)

Hierbei fällt jedoch auf, dass die Antworten nicht sehr stichhaltig sind. Dafür gibt es jedoch eine Lösung.

## Das Sprachmodell mit unseren Dokumenten 'verbinden'

Wir nutzen die Bibliothek *langchain* jetzt, um unsere Dokumente für KI-Modelle durchsuchbar zu machen und die am besten zur Frage passenden Texte mit an GPT-4 zu schicken. Dieses Vorgehen nennt sich *Retrieval Augmented Generation* (RAG) und ist eine der typischen Einsatzszenarien für Sprachmodelle.

Hier eine Veranschaulichung:

![Retrieval Augmented Generation](RAG.png)

Zunächst müssen wir unsere Texte in Langchain-Typen überführen:

In [ ]:
# Transformieren unserer Dokumente in Langchain Typen

from langchain_core.documents import Document

docs = [Document(doc) for doc in docs]

Als nächstes Stellen wir eine Verbindung zum Embedding-Modell her:

In [ ]:
# Embedding Model anlegen (so finden wir die besten Dokumente zur Frage)

from langchain_openai import AzureOpenAIEmbeddings

embedding_model = AzureOpenAIEmbeddings(
    api_version="2024-02-01",
    azure_deployment="embedding",
    api_key="", # Hier API-Key einfügen
    azure_endpoint="" # Hier Azure-Endpoint einfügen
)

Weiterhin benötigen wir eine Vektordatenbank, in der die Embeddings und Dokumente zusammen gespeichert werden, und welche uns eine performante Suche über diese bietet:

In [ ]:
# Vektordatenbank anlegen (dort werden die Embeddings gespeichert)

from langchain_community.vectorstores import FAISS

vectorstore = 

Diese Vektordatenbank verwenden wir dann auch gleich als *Retriever* für unser RAG und geben an, aufgrund welcher Kriterien gesucht werden soll: 

In [ ]:
# Vektordatenbank als Suchbasis (Retriever) nutzen

retriever = vectorstore.as_retriever(
    search_type="", 
    search_kwargs={"k": } # Die 3 am nächsten gelegenen Dokumente werden zurück geliefert
)

Wichtig ist auch der System Prompt, der die Anweisung an das Sprachmodell darstellt. 
Hierbei können wir steuern, wie die Antworten aussehen sollen. 

Außerdem bauen wir uns ein Prompt-Template aus dem System-Prompt, der Frage und dem gefundenden Kontext aus den Dokumenten zusammen:

In [ ]:
# System Prompt (Anweisung an LLM) anlegen

from langchain_core.prompts import ChatPromptTemplate

system_prompt = ""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "Frage: {question}"),
        ("system", "Kontext: {context}"),
    ]
)

Am Ende müssen wir die einzelnen Teile nur noch verketten. Daher leitet sich auch der Name *langchain* für *Lang(uage) Chain(ing)* ab:

In [ ]:
# Einzelne Komponenten verketten (Chain)

from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


chain = (
    
)

Diese Chain können wir nun wieder mit `invoke` ausführen, und die Antwort mit der vorherigen vergleichen:

In [ ]:
# Chain ausführen

rag_antwort = 

print("Frage: ", frage)
print("Antwort ohne RAG: ", antwort)
print("Antwort mit RAG: ", rag_antwort)

## Web-Interface starten

Die fertige Chain können wir mit 3 Zeilen Code sogar in ein [Gradio](https://www.gradio.app/) Web-Interface einbauen und mit unterschiedlichen Eingaben testen:

In [ ]:
# Chain in Webinterface einbinden

import gradio as gr

def answer_question(question, context):
    

iface = 

iface.launch()

In [ ]:
iface.close()

## Sprachinteraktion einbauen

Zuletzt können wir sogar Sprachinteraktion einbauen. Hierfür nutzen wir Speech-to-Text und Text-to-Speech-Dienste von Google, die zum Prototyping ohne Zugangsdaten nutzbar sind:

In [ ]:
import speech_recognition as sr
from gtts import gTTS

r = sr.Recognizer()

def answer_question(question):
    print(question)
    with sr.AudioFile(question) as source:
        audio = r.record(source)

    text = r.recognize_google(audio, language="de")
    answer = chain.invoke(text)
    print(answer)
    output = gTTS(answer, lang="de")
    output.save("output.mp3")
    return "./output.mp3", text, answer

iface = gr.Interface(
    fn=answer_question,
    inputs=gr.Microphone(type="filepath", label="Frage stellen"),
    outputs=[
        gr.Audio(type="filepath", label="Antwort als Audio"),
        gr.Textbox(label="Frage"),
        gr.Textbox(label="Antwort")
    ]
)

iface.launch()

In [ ]:
iface.close()